In [1]:
import numpy as np;
import pandas as pd;
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef,make_scorer
mccScorer = make_scorer(matthews_corrcoef)
from numpy import mean
from numpy import std

from google.colab import drive
drive.mount("/content/gdrive")

# ANOVA feature selection for numeric input and categorical output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.pipeline import Pipeline

from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

Mounted at /content/gdrive


In [2]:
#read training dataset 1
data1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_1_Training.csv')

X_train_actual_1=data1[:-2].T[1:].values

y_train_actual_1 = data1[-2:].T[-130:].T.values

In [3]:
#read training dataset 2
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_2_Training.csv')

X_train_actual_2=data2[:-4].T[1:].values

y_train_actual_2 = data2[-4:].T[-340:].T.values

In [4]:
co1={'X':X_train_actual_1,'Y': y_train_actual_1[0]};
co2={'X':X_train_actual_1,'Y': y_train_actual_1[1]};

co3={'X':X_train_actual_2,'Y': y_train_actual_2[0]};
co4={'X':X_train_actual_2,'Y': y_train_actual_2[1]};
co5={'X':X_train_actual_2,'Y': y_train_actual_2[2]};
co6={'X':X_train_actual_2,'Y': y_train_actual_2[3]};

In [11]:
def DT_HP_tuning(X,Y):
  # Instantiate a Decision Tree classifier: tree
  tree = DecisionTreeClassifier(class_weight='balanced',random_state=1)

  pipeline = Pipeline(
    [
     ('selector',SelectKBest(f_classif,X.shape[1]//2)),
     ('model',tree)
    ]
  )
  #"selector__k":[int(x) for x in np.linspace(start = 5 , stop = X.shape[1]//2, num = 50)],
  param_dist = {
              "model__max_features": [int(x) for x in np.linspace(start = 5 , stop = X.shape[1]//5, num = 50)],
              "model__min_samples_leaf": [int(x) for x in np.linspace(start = 5 , stop = X.shape[0]//10, num = 5)],
              "model__criterion": ["gini", "entropy"]
              }


  # Instantiate the GridSearchCV object
  search = GridSearchCV(pipeline, param_grid=param_dist,scoring=mccScorer, cv=3)

  # Fit it to the data
  result = search.fit(X,Y.astype('int'))

  # Print the tuned parameters and score
  print("Tuned Decision Tree Parameters: {}".format(result.best_params_))
  print("Best score is {}".format(result.best_score_))

In [8]:
DT_HP_tuning(**co1)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


Tuned Decision Tree Parameters: {'model__criterion': 'gini', 'model__max_features': 5686, 'model__min_samples_leaf': 11}
Best score is 0.43031730454687683


In [9]:
DT_HP_tuning(**co2)

Tuned Decision Tree Parameters: {'model__criterion': 'gini', 'model__max_features': 7504, 'model__min_samples_leaf': 13}
Best score is 0.476563439095226


In [12]:
DT_HP_tuning(**co3)

Tuned Decision Tree Parameters: {'model__criterion': 'gini', 'model__max_features': 674, 'model__min_samples_leaf': 5}
Best score is 0.2215794156360779


In [19]:
DT_HP_tuning(**co4)

Tuned Decision Tree Parameters: {'model__criterion': 'gini', 'model__max_features': 7589, 'model__min_samples_leaf': 5}
Best score is 0.18579338926531389


In [20]:
DT_HP_tuning(**co5)

Tuned Decision Tree Parameters: {'model__criterion': 'gini', 'model__max_features': 5581, 'model__min_samples_leaf': 12}
Best score is 0.8133447121349239


In [16]:
def feature_selector(X,Y,numberOfFeaturesToBeSelected=1):
  if(numberOfFeaturesToBeSelected == 1):
    totalNumberOfFeatures=X.shape[1]
    numberOfFeaturesToBeSelected=totalNumberOfFeatures//2;
  # define feature selection
  fs = SelectKBest(score_func=f_classif, k=numberOfFeaturesToBeSelected)
  # apply feature selection
  return fs.fit(X, Y)

In [21]:
def DecisionTree_with_cross_validation(X,Y,criterion,max_features,min_samples_leaf,numberOfFeatures=1):

  scoringReports={'mccScorer': mccScorer,'accuracy': 'accuracy','precision':'precision','recall':'recall', 'f1':'f1'}
  
  # define model
  model = DecisionTreeClassifier(criterion=criterion,max_features=max_features,min_samples_leaf=min_samples_leaf,class_weight='balanced',random_state=1)
  # define evaluation
  cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=1)

  X_selected_features=feature_selector(X,Y,numberOfFeaturesToBeSelected=numberOfFeatures).transform(X);
  scores = cross_validate(model,X_selected_features , Y.astype('int'), scoring=scoringReports, cv=cv)

  # summarize result
  print(model.get_params())
  print('MCC Score- Mean: %.3f SD: (%.3f)' % (mean(scores['test_mccScorer']), std(scores['test_mccScorer'])))
  print(list(scores.keys()))
  print("Means",mean(list(scores.values()),axis=1))
  print("SD",std(list(scores.values()),axis=1))

In [17]:
DecisionTree_with_cross_validation(co1['X'],co1['Y'],'gini',5686,11)

MCC Score- Mean: 0.112 SD: (0.092)
['fit_time', 'score_time', 'test_mccScorer', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1']
Means [0.12602425 0.01428938 0.11203548 0.63794926 0.33333333 0.39393939
 0.35217674]
SD [0.00177624 0.00330412 0.09162626 0.06049585 0.05892557 0.11338356
 0.06572667]


In [18]:
DecisionTree_with_cross_validation(co2['X'],co2['Y'],'gini',7504,13)

MCC Score- Mean: 0.385 SD: (0.139)
['fit_time', 'score_time', 'test_mccScorer', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1']
Means [0.1603775  0.01218851 0.38475151 0.68446089 0.5952381  0.75599129
 0.66480595]
SD [0.00159753 0.00065682 0.13901753 0.07726952 0.07009334 0.04775667
 0.05959143]
